In [ ]:
import gmsh
import sys
import os
import numpy as np
import import_ipynb
import matplotlib.pyplot as plt
from pathlib import Path

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))
from fem_processing import master_domain, gaussian_quadrature, matrices_assembly
from fem_pos_processing import graph_results

# `add_points()`

In [ ]:
def add_points(points, lc):
    point_tags = []
    for i, (x, y, z) in enumerate(points):
        tag = gmsh.model.geo.addPoint(x, y, z, lc, i + 1)
        point_tags.append(tag)
    return point_tags

# `add_lines()`

In [ ]:
def add_lines(point_tags):
    line_tags = []
    for i in range(len(point_tags)):
        start_point = point_tags[i]
        # a % b retorna o resto da divisão inteira de 'a' por 'b', quando a é maior que b;
        # ou retorna o próprio número 'a' quando ele é menor que 'b', pois a divisão inteira resulta em 0 e todo o valor de a se torna o resto;
        # ou retorna 0 quando 'a' é múltiplo de 'b'.
        end_point = point_tags[(i + 1) % len(point_tags)]  
        tag = gmsh.model.geo.addLine(start_point, end_point)
        line_tags.append(tag)
    return line_tags

# `add_surface()`

In [ ]:
def add_surface(line_tags):
    tag_curve = gmsh.model.geo.addCurveLoop(line_tags)
    tag_surface = gmsh.model.geo.addPlaneSurface([tag_curve])
    gmsh.model.geo.synchronize()
    return tag_surface

# `create_domain()`

In [ ]:
def create_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, h, auto_save=True, view_mesh=False):
    type, order = FINITE_ELEMENT
    vertices = [(-1, -1, 0), (-1, 1, 0), (1, 1, 0), (1, -1, 0)]

    # Define a new model
    gmsh.initialize()
    gmsh.model.add("vectorial_poisson")

    # Define the points of the domain.
    point_tags = add_points(vertices, h)

    # Criar linhas para formar as bordas do quadrado
    line_tags = add_lines(point_tags)
    
    # Criar um loop de linha e uma superfície plana
    free_space = add_surface(line_tags)

    # Adicionar grupos físicos para Dim=1
    for i, tag in enumerate([line_tags]):
        gmsh.model.addPhysicalGroup(1, tags=tag, tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])

    # Adicionar grupos físicos para Dim=2
    gmsh.model.addPhysicalGroup(2, [free_space], tag=MATERIAL[0]['tag'], name=MATERIAL[0]['name'])  
    
    # Gerar a malha 2D
    gmsh.option.setNumber("Mesh.SaveAll", 1)

    # Definir o tipo de elemento
    if type == 'Quadrangle':
        gmsh.model.mesh.setTransfiniteSurface(free_space, "Alternate")
        gmsh.model.mesh.setRecombine(2, free_space)

    # Define a ordem dos elementos
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(order)

    # Visualizar a malha no ambiente Gmsh (opcional)
    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/vectorial_poisson_domain_{type}{order}.msh"
        gmsh.write(file_path)
        gmsh.finalize()

# `apply_physics()`

In [ ]:
def apply_physics(FINITE_ELEMENT, mesh_data):
    """
    Adiciona uma nova chave 'source' a cada dicionário em conn_data.
    
    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    - element_type: Tuple (tipo do elemento, ordem).
    
    Retorna:
    - mesh_data: O dicionário atualizado com a chave 'source' em cada elemento de conn_data.
    """
    # Desempacotar o tipo de elemento
    type, order = FINITE_ELEMENT

    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']

    # Função forçante (fonte)
    def source_function(x, y):
        vector =  np.array([[np.cos(np.pi * x) * np.sin(np.pi * x)], 
                            [-np.sin(np.pi * x) * np.cos(np.pi * y)]])
        
        return (2 * np.pi ** 2 + 1) * vector

    for key, cell in cell_data.items():  
        # Adicionar as propriedades do materiais ao dicionário da célula
        cell['stiffness_term'] = []
        cell['mass_term'] = []
        cell['source'] = []
        
        # Calcular o valor de 'a' de acordo com a física do problema
        # mur = cell['material']['relative_magnetic_permeability']
        # er = cell['material']['relative_electric_permittivity']
        a = cell['material']['a_constant']
        
        # Get the global coordinates of the nodes
        ai = [nodes_data[idx]['xg'] for idx in cell['conn']]

        # Get the Gauss points and weights
        gauss_points, _ = gaussian_quadrature.gauss_data(FINITE_ELEMENT)

        # Adicionar a nova chave 'material' ao dicionário da célula
        for xik in gauss_points: 
            xg_e, yg_e = matrices_assembly.isomapping_to_global_coordinates(FINITE_ELEMENT, ai, xik)
            cell['stiffness_term'].append(a * np.eye(2))
            cell['mass_term'].append(1)           
        
            # Adicionar a fonte ao dicionário da célula
            cell['source'].append(source_function(xg_e, yg_e))

        # Dictionary with boundary nodes
        cell_data[key]['abc'] = {'type': None, 'conn_idx': None}

    return mesh_data

# `plot_shape_functions()`

In [ ]:
def plot_shape_functions_n0():
    """
    Plota as funções de forma vetoriais de Nedelec do tipo 1 no triângulo de referência.
    """
    # Definir uma grade de pontos dentro do triângulo de referência
    xi = np.linspace(0, 1, 10)
    eta = np.linspace(0, 1, 10)
    xi_grid, eta_grid = np.meshgrid(xi, eta)
    
    # Filtrar pontos dentro do triângulo de referência
    inside_triangle = xi_grid + eta_grid <= 1
    xi_grid = xi_grid[inside_triangle]
    eta_grid = eta_grid[inside_triangle]

    # Criar uma figura
    plt.figure(figsize=(16, 6))

    # Lista de cores
    colors = ['blue', 'red', 'green']
    
    # Iterar sobre cada função de forma
    for idx in range(3):
        plt.subplot(1, 3, idx + 1)  # Subplots para cada função de forma
        plt.title(fr'$\varphi_{idx+1}$')
        plt.xlabel(r'$\xi$')
        plt.ylabel(r'$\eta$')
        
        # Inicializar vetores para quiver
        u = np.zeros_like(xi_grid)
        v = np.zeros_like(eta_grid)
        
        # Calcular os vetores das funções de forma
        for i, (xi, eta) in enumerate(zip(xi_grid, eta_grid)):
            N = master_domain.shape_functions_n0(xi, eta)
            u[i], v[i] = N[idx][0, 0], N[idx][1, 0]
        
        # Plotar vetores com quiver
        plt.quiver(xi_grid, eta_grid, u, v,
                    angles='xy', scale_units='xy', scale=10, color=colors[idx]
        )
        
        # Plotar o triângulo de referência
        plt.plot([0, 1, 0, 0], [0, 0, 1, 0], 'k-')
        plt.axis('equal')
        plt.xlim(-0.2, 1.2)
    
    plt.tight_layout()
    plt.show()

# `set_analytical_solution()`

In [ ]:
def set_analytical_solution(x, y):
    """
    Calculate the vector solution u(x, y) and its gradient ∇u(x, y) in a 2D matrix form.

    Parameters:
    x (float or array-like): x-coordinate(s)
    y (float or array-like): y-coordinate(s)
    
    Returns:
    tuple:
        - u (ndarray): Vector solution u(x, y) as a 2D column matrix:
            [[cos(πx)sin(πx)],
             [-sin(πx)cos(πy)]]
        - grad_u (ndarray): Gradient ∇u(x, y), a 2x2 matrix with:
            [[∂u1/∂x, ∂u1/∂y],
             [∂u2/∂x, ∂u2/∂y]]
    """
    # Vector solution (2x1 matrix)
    u = np.array([[np.cos(np.pi * x) * np.sin(np.pi * x)],
                  [-np.sin(np.pi * x) * np.cos(np.pi * y)]])
    
    # Gradient of u (Jacobian matrix, 2x2)
    grad_u = np.array([[ np.pi * (np.cos(2 * np.pi * x) - 0.5),  0],
                       [-np.pi * np.cos(np.pi * x) * np.cos(np.pi * y),  np.pi * np.sin(np.pi * x) * np.sin(np.pi * y)]])
    
    return u, grad_u

# `plot_analytical_solution()`

In [ ]:
def plot_analytical_solution():
    """
    Plot the exact vector solution u(x, y) using quiver() over the domain [-1, 1]².
    The solution is defined as:
        u1(x, y) = cos(πx)sin(πx)
        u2(x, y) = -sin(πx)cos(πy)
    """
    # Define the exact solution
    x, y = np.meshgrid(
        np.linspace(-1, 1, 30),
        np.linspace(-1, 1, 30)
    )

    # Compute the exact solution components
    u_1 = np.cos(np.pi * x) * np.sin(np.pi * y)  # First component of u
    u_2 = -np.sin(np.pi * x) * np.cos(np.pi * y)  # Second component of u

    # Plot the vector field using quiver()
    plt.figure(figsize=(8, 8))
    plt.quiver(x, y, u_1, u_2,
                color='b', angles='xy', scale_units='xy', scale=15, width=0.003)
    plt.title("Exact Solution $\\mathbf{u}(x, y)$ over $[-1, 1]^2$")
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    plt.xlim([-1, 1])
    plt.ylim([-1, 1])
    plt.axis('equal') 
    plt.show()

# `plot_analytical_source()`

In [ ]:
def plot_analytical_source():
    """
    Plot the source vector f(x, y) using quiver() over the domain [-1, 1]².
    The solution is defined as:
        u1(x, y) = cos(πx)sin(πx)
        u2(x, y) = -sin(πx)cos(πy)
    """
    # Define the exact solution
    x, y = np.meshgrid(
        np.linspace(-1, 1, 30),
        np.linspace(-1, 1, 30)
    )

    # Compute the exact solution components
    # First component of u
    u_1 = (2*np.pi**2+1) * np.cos(np.pi * x) * np.sin(np.pi * x)  
    # Second component of u
    u_2 = -(2*np.pi**2+1) * np.sin(np.pi * x) * np.cos(np.pi * y)  

    # Plot the vector field using quiver()
    plt.figure(figsize=(8, 8))
    plt.quiver(x, y, u_1, u_2,
                color='r', angles='xy', scale_units='xy', scale=300, width=0.003)
    plt.title("Source $\\mathbf{f}(x, y)$ over $[-1, 1]^2$")
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    plt.xlim([-1, 1])
    plt.ylim([-1, 1])
    plt.axis('equal') 
    plt.show()